**Pandas Notebook 6**

A demonstration of advanced `pandas` syntax to accompany class Lecture.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

#More on `Groupby`





Try to predict the results of the `groupby` operation shown. The answer is below the image.
The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [4]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )
ds.head()



,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi


In [5]:
#Use groupby on index and get max of each group
ds.groupby(ds.index).max()


,x,y
A,3,hi
B,6,tx
C,9,sd


### `groupby.size` and `groupby.count()`

In [6]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [7]:
# Use groupby with size()

df_s = df.groupby("letter").size()
print(df_s)

letter
A    2
B    1
C    3
dtype: int64


`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [ ]:
# Use groupby with count()
df.groupby("letter").count()


,num,state
letter,,
A,2,1
B,1,1
C,2,2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [8]:
# Use value_count() on DataFrame described above
# Value_counts return Unique Values plus count of these values.


Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

In [ ]:
# groupby.size() show to tell only number of rows in each sub-group.

## Filtering by Group

In [10]:
# Let's read the elections dataset
elections = pd.read_csv("/content/elections.csv")
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


*Let*'s keep only the elections years where the maximum vote share `%` is less than 45%.

In [ ]:
# use filter function
filtered_elections = elections.groupby('Year').filter(lambda x: x['%'].max() < 45)
filtered_elections


,Year,Candidate,Party,Popular vote,Result,%
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408
24,1860,John Bell,Constitutional Union,590901,loss,12.639283
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
69,1912,William Taft,Republican,3486242,loss,23.218466
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
115,1968,George Wallace,American Independent,9901118,loss,13.571218


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [ ]:
# Use agg(max)
#Because when we apply multiply columns with different datatypes like int64,string.

,Candidate,%,Year,Result
Party,,,,
Democratic,Obama,52.9,2016,win
Independent,Perot,18.9,1996,loss
Republican,Trump,58.8,2016,win


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:


In [27]:
# Use sort and then groupby as describe in the lec
election_sort = elections.sort_values(by= "Year", ascending=False)
election_sort

election_sort.groupby("Party")[["Year","Candidate","Popular vote","%"]].agg(np.max).head(5)


<ipython-input-27-0bdcb142ed1d>:5: FutureWarning: The provided callable <function max at 0x7afbdd2cf880> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  election_sort.groupby("Party")[["Year","Candidate","Popular vote","%"]].agg(np.max).head(5)


,Year,Candidate,Popular vote,%
Party,,,,
American,1976,Thomas J. Anderson,873053,21.554001
American Independent,1976,Lester Maddox,9901118,13.571218
Anti-Masonic,1832,William Wirt,100715,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,1.335838
Citizens,1980,Barry Commoner,233052,0.270182


In [33]:
# grab first entry
election_sort.groupby("Party")[["Year","Candidate","Popular vote","%"]].first().agg(np.max).head(5)


<ipython-input-33-9eb4f44b66ed>:2: FutureWarning: The provided callable <function max at 0x7afbdd2cf880> is currently using DataFrame.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  election_sort.groupby("Party")[["Year","Candidate","Popular vote","%"]].first().agg(np.max).head(5)


,0
Year,2020
Candidate,Winfield Scott
Popular vote,81268924
%,54.951512


#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [45]:
# Use idxmax function
best_per_party = election_sort.loc[election_sort.groupby("Party")["%"].idxmax()]
best_per_party.head(3)

,Year,Candidate,Party,Popular vote,Result,%
22,1856,Millard Fillmore,American,873053,loss,21.554001
115,1968,George Wallace,American Independent,9901118,loss,13.571218
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583


In [47]:
# This is the computational part, Extract DataFrame based on above mentioned condition
best_per_party = election_sort.loc[election_sort.groupby("Party")["%"].idxmax()]
# Apply sort_index to match the formatting above
best_per_party = best_per_party.sort_index()
best_per_party.head(3)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [52]:

# Formatting, Again, the 2nd line is purely formatting:
best_per_party = election_sort.sort_values("%").drop_duplicates(["Party"],  keep="last")
best_per_party

,Year,Candidate,Party,Popular vote,Result,%
148,1996,John Hagelin,Natural Law,113670,loss,0.118219
164,2008,Chuck Baldwin,Constitution,199750,loss,0.152398
110,1956,T. Coleman Andrews,States' Rights,107929,loss,0.174883
147,1996,Howard Phillips,Taxpayers,184656,loss,0.192045
136,1988,Lenora Fulani,New Alliance,217221,loss,0.237804
89,1932,William Z. Foster,Communist,103307,loss,0.261069
127,1980,Barry Commoner,Citizens,233052,loss,0.270182
50,1896,John M. Palmer,National Democratic,134645,loss,0.969566
78,1920,Parley P. Christensen,Farmer–Labor,265398,loss,0.995804
42,1888,Alson Streeter,Union Labor,146602,loss,1.288861


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [53]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [62]:
# visualize all groups using .groups
grouped_by_party = elections.groupby("Party")
grouped_by_party.groups


{'American': [22, 126], 'American Independent': [115, 119, 124], 'Anti-Masonic': [6], 'Anti-Monopoly': [38], 'Citizens': [127], 'Communist': [89], 'Constitution': [160, 164, 172], 'Constitutional Union': [24], 'Democratic': [2, 4, 8, 10, 13, 14, 17, 20, 28, 29, 34, 37, 39, 45, 47, 52, 55, 57, 64, 70, 74, 77, 81, 83, 86, 91, 94, 97, 100, 105, 108, 111, 114, 116, 118, 123, 129, 134, 137, 140, 144, 151, 158, 162, 168, 176, 178], 'Democratic-Republican': [0, 1], 'Dixiecrat': [103], 'Farmer–Labor': [78], 'Free Soil': [15, 18], 'Green': [149, 155, 156, 165, 170, 177, 181], 'Greenback': [35], 'Independent': [121, 130, 143, 161, 167, 174], 'Liberal Republican': [31], 'Libertarian': [125, 128, 132, 138, 139, 146, 153, 159, 163, 169, 175, 180], 'National Democratic': [50], 'National Republican': [3, 5], 'National Union': [27], 'Natural Law': [148], 'New Alliance': [136], 'Northern Democratic': [26], 'Populist': [48, 61, 141], 'Progressive': [68, 82, 101, 107], 'Prohibition': [41, 44, 49, 51, 54,

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [64]:
# use get_group to extract 'Socialists' party results

grouped_by_party.get_group("Socialist")

,Year,Candidate,Party,Popular vote,Result,%
58,1904,Eugene V. Debs,Socialist,402810,loss,2.985897
62,1908,Eugene V. Debs,Socialist,420852,loss,2.850866
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
71,1916,Allan L. Benson,Socialist,590524,loss,3.194193
76,1920,Eugene V. Debs,Socialist,913693,loss,3.428282
85,1928,Norman Thomas,Socialist,267478,loss,0.728623
88,1932,Norman Thomas,Socialist,884885,loss,2.236211
92,1936,Norman Thomas,Socialist,187910,loss,0.412876
95,1940,Norman Thomas,Socialist,116599,loss,0.234237
102,1948,Norman Thomas,Socialist,139569,loss,0.286312


## Pivot Tables

### `Groupby` with multiple columns

We want to build a table showing the total number of babies born of each sex in each year. One way is to `groupby` using both columns of interest:

### `pivot_table`

In [91]:
import urllib.request
import os.path
import zipfile
import pandas as pd

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "C:\\Users\\Admin\\Downloads\\babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


In [92]:
# Find total count of baby names for both female and Male for each year using Pivot table
# babynames.groupby(["Year","Sex"])["Count"].agg(np.sum).head()

babynames_pivot = babynames.pivot_table(index="Year",
                                        values="Count",
                                        columns="Sex",
                                        aggfunc=np.sum)
babynames_pivot.head()

<ipython-input-92-3ceece3729f7>:4: FutureWarning: The provided callable <function sum at 0x7afbdd2cf250> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  babynames_pivot = babynames.pivot_table(index="Year",


Sex,F,M
Year,,
1910,5950,3213
1911,6602,3381
1912,9804,8142
1913,11860,10234
1914,13815,13111


### `pivot_table` with Multiple values

In [93]:
babynames.columns

Index(['State', 'Sex', 'Year', 'Name', 'Count'], dtype='object')

In [94]:
# Form a pivot table as describr in Lecture Slides
babynames_pivot = babynames.pivot_table(index="Year",
                                        values=["Count","State"],
                                        columns="Sex",
                                        aggfunc=np.max)
babynames_pivot.head(5)


<ipython-input-94-62646b12c91e>:2: FutureWarning: The provided callable <function max at 0x7afbdd2cf880> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  babynames_pivot = babynames.pivot_table(index="Year",


Count      State    
Sex      F    M     F   M
Year                     
1910   295  237    CA  CA
1911   390  214    CA  CA
1912   534  501    CA  CA
1913   584  614    CA  CA
1914   773  769    CA  CA

## Join Tables

What if we want to know the popularity of presidential candidates' first names in California in 2022? What can we do?

In [95]:
elections.head(10)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
9,1836,William Henry Harrison,Whig,550816,loss,37.721543


In [106]:
# Collect baby names for 2022

babynames_2022 = babynames[babynames["Year"]==2022]
babynames_2022.head()
babynames_2022["Name"].str.split().head()


,Name
235861,[Olivia]
235862,[Emma]
235863,[Camila]
235864,[Mia]
235865,[Sophia]


In [102]:
# Use split the candidate names in elections dataframe
elections["Candidate"].str.split().head()


,Candidate
0,"[Andrew, Jackson]"
1,"[John, Quincy, Adams]"
2,"[Andrew, Jackson]"
3,"[John, Quincy, Adams]"
4,"[Andrew, Jackson]"


`join` in pandas

In [107]:
#Merge both elections and babynames and report your analysis
merge_data= elections.merge(babynames,left_on="Candidate",right_on="Name")
merge_data


,Year_x,Candidate,Party,Popular vote,Result,%,State,Sex,Year_y,Name,Count


In [108]:
# Sort using Count
merge_data.sort_values(by="Count",ascending=False)


,Year_x,Candidate,Party,Popular vote,Result,%,State,Sex,Year_y,Name,Count
